In [13]:
import os, sys
import pandas as pd

from tab_gmb.dataset import Dataset
from tab_gmb.models import TabDDPMModel
from tab_gmb.tuner import RandomSearchTuner
from tab_gmb.experiment_runer import ExperimentRunner

In [14]:
test_dir = './titanic.csv'
test_data = pd.read_csv(test_dir)

dataset = Dataset(
    data=test_data,
    name='titanic',
    task_type='classification',
    target_column='Survived',
    num_columns=["Age", "Fare", "SibSp", "Parch"],
    cat_columns=["Pclass", "Sex", "Embarked"],
    verbose_init=True
)

Предупреждение: Колонки не классифицированы как числовые или категориальные: {'PassengerId', 'Ticket', 'Name', 'Cabin'}
Датасет 'titanic' успешно валидирован:
  - Размер: (712, 12)
  - Тип задачи: classification
  - Целевая колонка: Survived
  - Числовые колонки (4): ['Age', 'Fare', 'SibSp', 'Parch']
  - Категориальные колонки (3): ['Pclass', 'Sex', 'Embarked']


In [15]:
ddpm_space = {
    'batch_size': [4096],  # дискретный
    'lr': (3.5e-4, 9.2e-4, 1e-5),  # непрерывный с квантованием
    'num_timesteps': [1000],  # дискретный
    'n_layers_hidden': [2, 4, 6],  # дискретный
    'n_units_hidden': [256, 512, 1024],  # дискретный
    'transformation_num_type': ['None'],  # дискретный
    'transformation_cat_type': ['None']  # дискретный
}

In [16]:
model= TabDDPMModel()

In [17]:
tuner = RandomSearchTuner(n_trials=5)


runner = ExperimentRunner(
    dataset=dataset,
    model=model,
    tuner=tuner,
    hyperparameter_space=ddpm_space,
    metrics=['c2st', 'ml_efficacy', 'pair', 'shape'],
    metric_weights={'c2st': 2.0, 'ml_efficacy': -1.0, 'pair': -1.0, 'shape': -1.0},
    n_folds=3
)


results = runner.run_experiment()

--------------------------------------------------   
{'batch_size': 4096, 'lr': 0.0007400000000000001, 'n_layers_hidden': 2, 'n_units_hidden': 1024, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
-0.24194720567091765                                 
{'c2st': 0.992859937402191, 'ml_efficacy': 0.6815664893426127, 'pair': 0.7751506921857665, 'shape': 0.7709498989469207}
--------------------------------------------------   
--------------------------------------------------                                
{'batch_size': 4096, 'lr': 0.0006600000000000001, 'n_layers_hidden': 2, 'n_units_hidden': 512, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
-0.892074951319614                                                                
{'c2st': 0.897039645279082, 'ml_efficacy': 0.9751744353751043, 'pair': 0.844101578418191, 'shape': 0.8668782280844827}
--------------------------------------------------      

In [18]:
print('Лучшие параметры:', results['best_hyperparameters'])
print('Лучший score:', results['best_score'])
print(results['best_com_score'])

Лучшие параметры: {'batch_size': 4096, 'lr': 0.00076, 'n_layers_hidden': 4, 'n_units_hidden': 256, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
Лучший score: -1.1084599800507884
{'c2st': 0.810641627543036, 'ml_efficacy': 0.9051999823919723, 'pair': 0.9124966619566742, 'shape': 0.912046590788214}
